In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

class SiameseDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        logger.info(f"Loading CSV: {csv_file}")
        self.pairs_df = pd.read_csv(csv_file)
        self.transform = transform
        self.valid_indices = self._validate_paths()
        logger.info(f"Found {len(self.valid_indices)} valid pairs out of {len(self.pairs_df)}")
    
    def _validate_paths(self):
        valid_indices = []
        for idx in range(len(self.pairs_df)):
            id_img_path = self.pairs_df.iloc[idx]['id_image_path']
            person_img_path = self.pairs_df.iloc[idx]['person_image_path']
            try:
                if not os.path.isfile(id_img_path) or not os.path.isfile(person_img_path):
                    logger.warning(f"Invalid paths at index {idx}: {id_img_path}, {person_img_path}")
                    continue
                Image.open(id_img_path).convert('RGB').close()
                Image.open(person_img_path).convert('RGB').close()
                valid_indices.append(idx)
            except Exception as e:
                logger.warning(f"Cannot read images at index {idx}: {id_img_path}, {person_img_path} - {e}")
        return valid_indices
    
    def __len__(self):
        return len(self.valid_indices)
    
    def __getitem__(self, idx):
        actual_idx = self.valid_indices[idx]
        id_img_path = self.pairs_df.iloc[actual_idx]['id_image_path']
        person_img_path = self.pairs_df.iloc[actual_idx]['person_image_path']
        label = self.pairs_df.iloc[actual_idx]['label']
        
        try:
            id_img = Image.open(id_img_path).convert('RGB')
            person_img = Image.open(person_img_path).convert('RGB')
        except Exception as e:
            logger.error(f"Error loading images at index {actual_idx}: {id_img_path}, {person_img_path} - {e}")
            return None
        
        if self.transform:
            try:
                id_img = self.transform(id_img)
                person_img = self.transform(person_img)
            except Exception as e:
                logger.error(f"Error applying transform at index {actual_idx}: {e}")
                return None
        
        return id_img, person_img, torch.tensor(label, dtype=torch.float32)

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.backbone = models.resnet18(pretrained=True)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()
        self.head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128)
        )
    
    def forward_one(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x
    
    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
    
    def forward(self, output1, output2, label):
        euclidean_distance = torch.nn.functional.pairwise_distance(output1, output2)
        loss_same = label * torch.pow(euclidean_distance, 2)
        loss_diff = (1 - label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
        loss = torch.mean(loss_same + loss_diff)
        return loss, euclidean_distance

def train_and_validate(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, output_dir):
    epoch_losses = []
    matching_distances = []
    non_matching_distances = []
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        batch_count = 0
        total_batches = len(train_loader)
        logger.info(f"Epoch {epoch+1}/{num_epochs}, {total_batches} batches")
        
        for batch_idx, batch in enumerate(train_loader):
            if batch is None:
                logger.warning(f"Skipping invalid batch {batch_idx+1}/{total_batches}")
                continue
            
            img1, img2, label = batch
            img1, img2, label = img1.to(device), img2.to(device), label.to(device)
            
            optimizer.zero_grad()
            output1, output2 = model(img1, img2)
            loss, _ = criterion(output1, output2, label)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * img1.size(0)
            batch_count += img1.size(0)
            
            if (batch_idx + 1) % 20 == 0 or (batch_idx + 1) == total_batches:
                logger.info(f"Batch {batch_idx+1}/{total_batches}, Batch Loss: {loss.item():.4f}")
        
        if batch_count == 0:
            logger.error("No valid batches processed in epoch")
            return None, None, None
        
        epoch_loss = running_loss / batch_count
        epoch_losses.append(epoch_loss)
        logger.info(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}")
        
        model.eval()
        matching_dist = []
        non_matching_dist = []
        with torch.no_grad():
            for batch in val_loader:
                if batch is None:
                    continue
                img1, img2, label = batch
                img1, img2, label = img1.to(device), img2.to(device), label.to(device)
                output1, output2 = model(img1, img2)
                _, dist = criterion(output1, output2, label)
                for d, l in zip(dist.cpu().numpy(), label.cpu().numpy()):
                    if l == 1:
                        matching_dist.append(d)
                    else:
                        non_matching_dist.append(d)
        
        mean_matching = np.mean(matching_dist) if matching_dist else float('inf')
        mean_non_matching = np.mean(non_matching_dist) if non_matching_dist else 0.0
        gap = mean_non_matching - mean_matching
        matching_distances.append(mean_matching)
        non_matching_distances.append(mean_non_matching)
        logger.info(f"Validation: Matching Dist: {mean_matching:.4f}, Non-Matching Dist: {mean_non_matching:.4f}, Gap: {gap:.4f}")
        
        scheduler.step(epoch_loss)
    
    model_path = os.path.join(output_dir, 'siamese_model_final.pth')
    torch.save(model.state_dict(), model_path)
    logger.info(f"Saved final model at {model_path}")
    
    epochs = range(1, num_epochs + 1)
    plt.figure(figsize=(10, 6))
    plt.bar(epochs, matching_distances, width=0.4, label='Matching Distance', color='blue', alpha=0.5)
    plt.bar([e + 0.4 for e in epochs], non_matching_distances, width=0.4, label='Non-Matching Distance', color='red', alpha=0.5)
    plt.xlabel('Epoch')
    plt.ylabel('Distance')
    plt.title('Matching vs Non-Matching Distances per Epoch')
    plt.legend()
    plt.grid(True)
    plot_path = os.path.join(output_dir, 'distance_plot1.png')
    plt.savefig(plot_path)
    plt.close()
    logger.info(f"Saved distance plot at {plot_path}")
    
    return epoch_losses, matching_distances, non_matching_distances

def main():
    train_csv = r"D:\Projects\PhotosWorkl\train_pairs_balanced.csv"
    test_csv = r"D:\Projects\PhotosWorkl\test_pairs_balanced.csv"
    output_dir = r"D:\Projects\PhotosWorkl\\"
    batch_size = 32
    num_epochs = 15
    learning_rate = 0.0005
    weight_decay = 0.0001
    
    os.makedirs(output_dir, exist_ok=True)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")
    
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    logger.info("Loading datasets")
    train_dataset = SiameseDataset(train_csv, transform=train_transform)
    test_dataset = SiameseDataset(test_csv, transform=test_transform)
    
    if len(train_dataset) == 0:
        logger.error("No valid training pairs. Check CSV paths and image accessibility.")
        return
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,  
        pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True
    )
    
    logger.info("Initializing model")
    model = SiameseNetwork().to(device)
    criterion = ContrastiveLoss(margin=1.0)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    logger.info("Starting training")
    epoch_losses, matching_distances, non_matching_distances = train_and_validate(
        model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs, device, output_dir
    )
    
    if epoch_losses is not None:
        logger.info("Training completed")
    else:
        logger.error("Training failed. Check logs for details.")



In [10]:
if __name__ == "__main__":
    main()

2025-05-22 16:40:44,173 - INFO - Using device: cuda
2025-05-22 16:40:44,174 - INFO - Loading datasets
2025-05-22 16:40:44,175 - INFO - Loading CSV: D:\Projects\PhotosWorkl\train_pairs_balanced.csv
2025-05-22 16:40:47,593 - INFO - Found 4658 valid pairs out of 4658
2025-05-22 16:40:47,594 - INFO - Loading CSV: D:\Projects\PhotosWorkl\test_pairs_balanced.csv
2025-05-22 16:40:48,443 - INFO - Found 1166 valid pairs out of 1166
2025-05-22 16:40:48,444 - INFO - Initializing model
2025-05-22 16:40:48,593 - INFO - Starting training
2025-05-22 16:40:48,594 - INFO - Epoch 1/15, 146 batches


C:\Users\Vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Vlad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\optim\lr_scheduler.py:62: UserWarning

2025-05-22 16:41:02,527 - INFO - Batch 20/146, Batch Loss: 0.2318
2025-05-22 16:41:13,621 - INFO - Batch 40/146, Batch Loss: 0.2282
2025-05-22 16:41:24,960 - INFO - Batch 60/146, Batch Loss: 0.3668
2025-05-22 16:41:36,432 - INFO - Batch 80/146, Batch Loss: 0.2189
2025-05-22 16:41:47,703 - INFO - Batch 100/146, Batch Loss: 0.2916
2025-05-22 16:41:59,415 - INFO - Batch 120/146, Batch Loss: 0.3006
2025-05-22 16:42:11,830 - INFO - Batch 140/146, Batch Loss: 0.3113
2025-05-22 16:42:15,547 - INFO - Batch 146/146, Batch Loss: 0.2492
2025-05-22 16:42:15,548 - INFO - Epoch 1/15, Train Loss: 0.4252
2025-05-22 16:42:22,030 - INFO - Validation: Matching Dist: 0.1139, Non-Matching Dist: 0.1164, Gap: 0.0025
2025-05-22 16:42:22,032 - INFO - Epoch 2/15, 146 batches
2025-05-22 16:42:33,304 - INFO - Batch 20/146, Batch Loss: 0.2365
2025-05-22 16:42:44,768 - INFO - Batch 40/146, Batch Loss: 0.2735
2025-05-22 16:42:58,524 - INFO - Batch 60/146, Batch Loss: 0.2971
2025-05-22 16:43:09,479 - INFO - Batch 80/

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import logging
import tkinter as tk
from tkinter import ttk, messagebox
from torch.utils.data import Dataset, DataLoader

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.backbone = models.resnet18(pretrained=True)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()
        self.head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128)
        )
    
    def forward_one(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x
    
    def forward(self, input1, input2):
        output1 = self.forward_one(input1)
        output2 = self.forward_one(input2)
        return output1, output2

class SiameseDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        logger.info(f"Loading evaluation CSV: {csv_file}")
        self.pairs_df = pd.read_csv(csv_file)
        self.transform = transform
        self.valid_indices = self._validate_paths()
        logger.info(f"Found {len(self.valid_indices)} valid pairs out of {len(self.pairs_df)}")
    
    def _validate_paths(self):
        valid_indices = []
        for idx in range(len(self.pairs_df)):
            id_img_path = self.pairs_df.iloc[idx]['id_image_path']
            person_img_path = self.pairs_df.iloc[idx]['person_image_path']
            try:
                if not os.path.isfile(id_img_path) or not os.path.isfile(person_img_path):
                    logger.warning(f"Invalid paths at index {idx}: {id_img_path}, {person_img_path}")
                    continue
                Image.open(id_img_path).convert('RGB').close()
                Image.open(person_img_path).convert('RGB').close()
                valid_indices.append(idx)
            except Exception as e:
                logger.warning(f"Cannot read images at index {idx}: {id_img_path}, {person_img_path} - {e}")
        return valid_indices
    
    def __len__(self):
        return len(self.valid_indices)
    
    def __getitem__(self, idx):
        actual_idx = self.valid_indices[idx]
        id_img_path = self.pairs_df.iloc[actual_idx]['id_image_path']
        person_img_path = self.pairs_df.iloc[actual_idx]['person_image_path']
        label = self.pairs_df.iloc[actual_idx]['label']
        
        try:
            id_img = Image.open(id_img_path).convert('RGB')
            person_img = Image.open(person_img_path).convert('RGB')
        except Exception as e:
            logger.error(f"Error loading images at index {actual_idx}: {id_img_path}, {person_img_path} - {e}")
            return None
        
        if self.transform:
            try:
                id_img = self.transform(id_img)
                person_img = self.transform(person_img)
            except Exception as e:
                logger.error(f"Error applying transform at index {actual_idx}: {e}")
                return None
        
        return id_img, person_img, torch.tensor(label, dtype=torch.float32), id_img_path, person_img_path

def evaluate_model(model, test_loader, device, output_dir, thresholds=[0.25, 0.3, 0.35, 0.4]):
    logger.info("Evaluating model on test set")
    model.eval()
    matching_distances = []
    non_matching_distances = []
    distances = []
    labels = []
    
    with torch.no_grad():
        for batch in test_loader:
            if batch is None:
                continue
            img1, img2, label, _, _ = batch
            img1, img2, label = img1.to(device), img2.to(device), label.to(device)
            output1, output2 = model(img1, img2)
            dist = torch.nn.functional.pairwise_distance(output1, output2)
            distances.extend(dist.cpu().numpy())
            labels.extend(label.cpu().numpy())
            for d, l in zip(dist.cpu().numpy(), label.cpu().numpy()):
                if l == 1:
                    matching_distances.append(d)
                else:
                    non_matching_distances.append(d)
    
    mean_matching = np.mean(matching_distances) if matching_distances else float('inf')
    mean_non_matching = np.mean(non_matching_distances) if non_matching_distances else 0.0
    gap = mean_non_matching - mean_matching
    logger.info(f"Test Set - Matching Dist: {mean_matching:.4f}, Non-Matching Dist: {mean_non_matching:.4f}, Gap: {gap:.4f}")
    
    for threshold in thresholds:
        predictions = [1 if d < threshold else 0 for d in distances]
        accuracy = np.mean([1 if p == l else 0 for p, l in zip(predictions, labels)])
        logger.info(f"Threshold {threshold:.2f}: Accuracy {accuracy:.4f}")
    
    plt.figure(figsize=(10, 6))
    plt.hist(matching_distances, bins=50, alpha=0.5, label='Matching', color='blue')
    plt.hist(non_matching_distances, bins=50, alpha=0.5, label='Non-Matching', color='red')
    plt.xlabel('Distance')
    plt.ylabel('Frequency')
    plt.title('Distance Histogram: Matching vs Non-Matching Pairs')
    plt.legend()
    plt.grid(True)
    plot_path = os.path.join(output_dir, 'distance_histogram.png')
    plt.savefig(plot_path)
    plt.close()
    logger.info(f"Saved distance histogram at {plot_path}")
    
    return mean_matching, mean_non_matching, gap, distances, labels

class SiameseGUI:
    def __init__(self, root, model, transform, device):
        self.root = root
        self.model = model
        self.transform = transform
        self.device = device
        self.root.title("Siamese Network Manual Testing")
        
        tk.Label(root, text="ID Image Path:").grid(row=0, column=0, padx=5, pady=5)
        self.id_path_entry = tk.Entry(root, width=50)
        self.id_path_entry.grid(row=0, column=1, padx=5, pady=5)
        
        tk.Label(root, text="Person Image Path:").grid(row=1, column=0, padx=5, pady=5)
        self.person_path_entry = tk.Entry(root, width=50)
        self.person_path_entry.grid(row=1, column=1, padx=5, pady=5)
        
        tk.Label(root, text="True Label (0/1, optional):").grid(row=2, column=0, padx=5, pady=5)
        self.label_entry = tk.Entry(root, width=10)
        self.label_entry.grid(row=2, column=1, padx=5, pady=5, sticky='w')
        
        tk.Label(root, text="Threshold:").grid(row=3, column=0, padx=5, pady=5)
        self.threshold_entry = tk.Entry(root, width=10)
        self.threshold_entry.grid(row=3, column=1, padx=5, pady=5, sticky='w')
        self.threshold_entry.insert(0, "0.3")
        
        self.id_image_label = tk.Label(root)
        self.id_image_label.grid(row=4, column=0, padx=5, pady=5)
        self.person_image_label = tk.Label(root)
        self.person_image_label.grid(row=4, column=1, padx=5, pady=5)
        
        self.result_label = tk.Label(root, text="", font=("Arial", 12))
        self.result_label.grid(row=5, column=0, columnspan=2, padx=5, pady=5)
        
        tk.Button(root, text="Test Pair", command=self.test_pair).grid(row=6, column=0, columnspan=2, pady=10)
        tk.Button(root, text="Clear", command=self.clear).grid(row=7, column=0, columnspan=2, pady=5)
    
    def test_pair(self):
        id_path = self.id_path_entry.get().strip()
        person_path = self.person_path_entry.get().strip()
        threshold = self.threshold_entry.get().strip()
        label = self.label_entry.get().strip()
        
        try:
            threshold = float(threshold)
            if threshold <= 0:
                raise ValueError("Threshold must be positive")
        except ValueError:
            messagebox.showerror("Error", "Invalid threshold. Enter a positive number (e.g., 0.3).")
            return
        
        if not os.path.isfile(id_path) or not os.path.isfile(person_path):
            messagebox.showerror("Error", "Invalid image paths. Check files exist.")
            return
        
        try:
            id_img = Image.open(id_path).convert('RGB')
            person_img = Image.open(person_path).convert('RGB')
        except Exception as e:
            messagebox.showerror("Error", f"Cannot load images: {e}")
            return
        
        id_img_tk = id_img.resize((224, 224))
        person_img_tk = person_img.resize((224, 224))
        self.id_photo = ImageTk.PhotoImage(id_img_tk)
        self.person_photo = ImageTk.PhotoImage(person_img_tk)
        self.id_image_label.config(image=self.id_photo)
        self.person_image_label.config(image=self.person_photo)
        
        try:
            id_tensor = self.transform(id_img).unsqueeze(0).to(self.device)
            person_tensor = self.transform(person_img).unsqueeze(0).to(self.device)
            self.model.eval()
            with torch.no_grad():
                output1, output2 = self.model(id_tensor, person_tensor)
                distance = torch.nn.functional.pairwise_distance(output1, output2).item()
        except Exception as e:
            messagebox.showerror("Error", f"Error computing distance: {e}")
            return
        
        prediction = "Matching" if distance < threshold else "Non-Matching"
        result_text = f"Distance: {distance:.4f}\nPrediction: {prediction} (Threshold: {threshold:.2f})"
        if label:
            try:
                label = int(label)
                if label in [0, 1]:
                    result_text += f"\nTrue Label: {'Matching' if label == 1 else 'Non-Matching'}"
                    result_text += f"\nCorrect: {((distance < threshold) == (label == 1))}"
            except ValueError:
                result_text += "\nInvalid true label (use 0 or 1)"
        
        self.result_label.config(text=result_text)
        logger.info(f"Manual test - ID: {id_path}, Person: {person_path}, Distance: {distance:.4f}, Prediction: {prediction}")
    
    def clear(self):
        self.id_path_entry.delete(0, tk.END)
        self.person_path_entry.delete(0, tk.END)
        self.label_entry.delete(0, tk.END)
        self.threshold_entry.delete(0, tk.END)
        self.threshold_entry.insert(0, "0.3")
        self.id_image_label.config(image='')
        self.person_image_label.config(image='')
        self.result_label.config(text='')

def main():
    # Configuration
    model_path = r"D:\Projects\PhotosWorkl\siamese_model_final.pth"
    test_csv = r"D:\Projects\PhotosWorkl\test_pairs_balanced.csv"
    output_dir = r"D:\Projects\PhotosWorkl\MoreOutputs"
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    # Load model
    logger.info("Loading model")
    model = SiameseNetwork().to(device)
    try:
        model.load_state_dict(torch.load(model_path, map_location=device))
    except Exception as e:
        logger.error(f"Error loading model: {e}")
        return
    
    # Evaluate on test set
    test_dataset = SiameseDataset(test_csv, transform=transform)
    if len(test_dataset) == 0:
        logger.error("No valid test pairs. Check CSV paths and image accessibility.")
        return
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=32,
        shuffle=False,
        num_workers=0,
        pin_memory=True
    )
    
    mean_matching, mean_non_matching, gap, distances, labels = evaluate_model(
        model, test_loader, device, output_dir, thresholds=[0.25, 0.3, 0.35, 0.4]
    )
    
    root = tk.Tk()
    app = SiameseGUI(root, model, transform, device)
    root.mainloop()

if __name__ == "__main__":
    from PIL import ImageTk
    main()

2025-05-22 16:29:37,672 - INFO - Using device: cuda
2025-05-22 16:29:37,673 - INFO - Loading model
2025-05-22 16:29:37,889 - INFO - Loading evaluation CSV: D:\Projects\PhotosWorkl\test_pairs_balanced.csv


C:\Users\Vlad\AppData\Local\Temp\ipykernel_28272\3880791638.py:283: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=d

2025-05-22 16:29:38,754 - INFO - Found 1166 valid pairs out of 1166
2025-05-22 16:29:38,756 - INFO - Evaluating model on test set
2025-05-22 16:29:46,636 - INFO - Test Set - Matching Dist: 0.2871, Non-Matching Dist: 0.3335, Gap: 0.0464
2025-05-22 16:29:46,640 - INFO - Threshold 0.25: Accuracy 0.5858
2025-05-22 16:29:46,644 - INFO - Threshold 0.30: Accuracy 0.5669
2025-05-22 16:29:46,649 - INFO - Threshold 0.35: Accuracy 0.5386
2025-05-22 16:29:46,653 - INFO - Threshold 0.40: Accuracy 0.5420
2025-05-22 16:29:47,143 - INFO - Saved distance histogram at D:\Projects\PhotosWorkl\MoreOutputs\distance_histogram.png
2025-05-22 16:30:27,303 - INFO - Manual test - ID: D:/Projects/PhotosWorkl/extracted_id_faces/person_706_id_face.jpg, Person: D:/Projects/finalGPT/ORIGINALS_FACE_EXTRACTIONS_128/863/863-3.jpg, Distance: 0.2531, Prediction: Matching
2025-05-22 16:30:55,638 - INFO - Manual test - ID: D:/Projects/PhotosWorkl/extracted_id_faces/person_364_id_face.jpg, Person: D:/Projects/finalGPT/ORIGI